O que fazer nesse arquivo:
V2: 
- retirada de Bag of Words, porque TfIdf é uma melhoria dele.
- retirada dos grupos de menor expressividade nos dados.

- Consistência e estabilidade: verificar a consistência dos resultados de agrupamentos em diferentes execuções ou subconjuntos de dados.
- Análise de sensibilidade: Avaliar como pequenas alterações nos dados afetam os resultados do agrupamentos. Modelos robustos a pequenas mudanças são preferíveis
- Interpetação e Relevância os domínio: considerar qual técnica faz mais sentidos para o domínio.
>- obs: Word2Vec pode ser melhor para textos maiores e mais contextualizados, equanto  TF-IDF pode funcionar melhor para textos mennores e mais diretos
- Pode ser necessário dividir os arquivos para não estourar a RAM google:
>- exemplo: uma para visualização e limpeza, outros para aplicaddo de técnicas diferentes de tratamento e um outro para a junção das técnicas.
- verificar se precisa de redução de dimensão
- aplicar nomavente

Porque fazer isso?
- Aproveitar as vantagens os algoritmos redutores para melhorar a qualidade dos agrupamentos
-

#### Imports

In [ ]:
# imports 

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.stem import WordNetLemmatizer
import re
import nltk
from nltk.corpus import wordnet,stopwords
from nltk import pos_tag
from collections import Counter
import regex as re
from sklearn.metrics import adjusted_rand_score, f1_score, precision_score, recall_score, homogeneity_completeness_v_measure
import umap

#### Funções

In [ ]:
# Função para plotar a distribuição dos pontos 
def plot_(n_components,scaled,df_nome,y_l):
    # plt.scatter(scaled[:,0],scaled[:,1],c=y_l, cmap='viridis',s=2)
    plt.scatter(scaled[:,0],scaled[:,1],c=y_l,s=2)
    plt.title(f' {df_nome}')
    plt.xlabel('Comp  1')
    plt.ylabel('Comp  2' if n_components > 1 else '')
    plt.grid()


In [ ]:
# Função para calcular as métricas 

metrica = []
def coleta_metricas(y,labels,dataset_name):
  ari_score = adjusted_rand_score(y, labels)
  metricas.append([dataset_name,'ARI Score',ari_score])
  #={'dataset':dataset, 'metrica':'ARI Score df','score':ari_score }
  # Para V-measure
  homogeneity, completeness, v_measure = homogeneity_completeness_v_measure(y, labels)
  metricas.append([dataset_name,'homogeneity',homogeneity])
  metricas.append([dataset_name,'completeness',completeness])
  metricas.append([dataset_name, 'v_measure',v_measure])

  # Para F1-Score, Precisão e Revocação  (adaptar para multiclasse)
  precision = precision_score(y, labels, average='macro')
  recall = recall_score(y, labels, average='macro')
  f1_scr = f1_score(y, labels, average='macro')
  metricas.append([dataset_name, 'precision',precision])
  metricas.append([dataset_name,'recall',recall])
  metricas.append([dataset_name, 'f1_score',f1_scr])

  return metrica

In [ ]:
# Função para gráfico
def plot(sil_avg, best_k, df_name, marker='o'):
  plt.scatter( sil_avg.iloc[:,0],sil_avg.iloc[:,1], marker=marker, s=2)
  title = f'Melhor k: {df_name} - {best_k}' if isinstance(best_k, int) else f'dataset: {df}'
  plt.title(title)
  plt.xlabel('Clusters (k)')
  plt.ylabel('Escore')
  plt.grid()


In [ ]:
# df.head()
import pandas as pd

# Calculando o comprimento de cada texto
df['num_palavras'] = df['text'].apply(lambda x: len(str(x).split()))

# Encontrando o número de palavras do maior e do menor texto
num_palavras_maximo = df['num_palavras'].max()
num_palavras_minimo = df['num_palavras'].min()

print(f'O maior texto tem: {num_palavras_maximo} palavras')
print(f'O menor texto tem: {num_palavras_minimo} palavras')

In [ ]:
#### Coleta dos dados


##### DF1

In [ ]:
# !pip install --upgrade pyarrow

In [ ]:
# %run open_df1.ipynb

In [ ]:
# #Limpeza adicional do df1

# # extraindo partes do texto sem significado no texto, mas que podem impactar o agrupamento
# qtd = 0
# total_pre = df_1.shape[0]
# import regex as re
# qtd = qtd + df_1['text'].str.contains('confirmed their accuracy').sum() + df_1['text'].str.contains('\(Reuters\) - ').sum() + df_1['text'].str.contains('@realDonaldTrump: - ').sum()
# df_1['text'] = df_1['text'].str.replace(r'^.*\(Reuters\) - ', '', regex=True)
# # df_1.loc[:, 'text'] = df_1['text'].str.replace(r'^.*\(Reuters\) - ', '', regex=True)
# df_1['text'] = df_1['text'].str.replace(r'@realDonaldTrump\s*:\s*-\s*', '', regex=True, flags=re.IGNORECASE)
# # df_1.loc[:, 'text'] = df_1['text'].str.replace(r'@realDonaldTrump\s*:\s*-\s*', '', regex=True, flags=re.IGNORECASE)
# df_1['text'] = df_1['text'].str.replace(r'^.*?confirmed their accuracy\.', '', regex=True)
# print(f'{qtd} atualizados')

# # limpando ocorrências de texto vazias:
# df_1 = df_1[~(df_1['text'].str.strip() == '')]

# # extraindo duplicatas:
# # de todos os atributos
# qtd = qtd + df_1.duplicated().sum()
# print(f'shape antes da exclusão de duplicatas',df_1.shape)
# df_1.drop_duplicates(inplace=True)

# # de textos duplicados
# qtd = qtd+ df_1.duplicated(subset=['text']).sum()
# df_1.drop_duplicates(subset=['text'],inplace=True)
# print(f'shape após da exclusão de duplicatas',df_1.shape)
# total_pos = df_1.shape[0]

In [ ]:
# print(f'Número de correções entre limpeza e exclusões:{qtd}, que representa {round(qtd/total_pre*100,2)}% ocorrências com os problemas indicados' )

In [ ]:
#Análise da frequência da palavras
# all_texts = ' '.join(df['txt_lmtzd'].tolist())

# # Tokenização
# tokens = nltk.word_tokenize(all_texts)

# # Remoção de stop words (ajuste o idioma conforme necessário)
# stop_words = set(stopwords.words('portuguese'))
# tokens_without_sw = [word for word in tokens if not word in stop_words]
# # Frequência de palavras
# word_freq = Counter(tokens_without_sw)

# # Exibindo as 10 palavras mais comuns
# most_common_words = word_freq.most_common(200)

# # print("As 150 palavras mais comuns e suas frequências:")
# # for word, freq in most_common_words:
# #     print(f"{word}: {freq}")

In [ ]:
# df = df_1.copy()

##### DF2

In [ ]:
%run open_df2.ipynb

In [ ]:
# Excluindo duplicatas
print(f'Com duplicadas: {df_2.shape[0]}')
df_2.drop_duplicates(inplace=True)
df_2.drop_duplicates(subset=['text'], inplace=True)
print(f'Sem duplicadas: {df_2.shape[0]}')

In [ ]:
# df = df_1.copy()
df = df_2.copy()
y_df = df['target']

In [ ]:
df['subject'].value_counts()

In [ ]:
# print('Porporção por alvo')
# round(df.groupby(['target'])['text'].count()/df.shape[0]*100,2)

In [ ]:
# Padronização do target
label_encoder = LabelEncoder()
df['target'] = label_encoder.fit_transform(df['subject'])
df.head()

In [ ]:
df['target'].unique(), df['subject'].unique()

##### Data_dataset


### Pré-Processamento

A limpeza e pré-processamento consistiu no seguinte:

1. conversão para minúsculas
2. remoção de caracteres não alfa
3. remoção de palavras isoladas e espaços extras, remoção de números;
4. remoção de stowords
5. Lemtização e Stemming
6. repetição dos passos 2 e 3 para remoção de ruídos resultantes do passo 5
7. remoção de duplicatas
8. remoção de frases referêntes ás fontes das informações

In [ ]:
# 2 - pré-processamento dos textos
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.stem import WordNetLemmatizer
import re
import nltk
from nltk.corpus import wordnet,stopwords
from nltk import pos_tag
from collections import Counter

In [ ]:
#Só precisa baixar uma vez, se for máquina local
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
# nltk.download('punkt')
# nltk.download('stopwords')




In [ ]:
stop_words = stopwords.words('english')

In [ ]:
# processo de lemmatização:
# remoção de caracteres não alfa e conversão para minúsculas,
# remoção de palavras isoladas e espaços extras, remoção de números;
# remoção de stowords


lemmatizer = WordNetLemmatizer()

def preprocess(text):
    # Removendo caracteres não alfabéticos e convertendo para minúsculas
    text = re.sub(r'\W', ' ', str(text))
    text = text.lower()

    # Removendo palavras isoladas e espaços extras
    text = re.sub(r'\s+[a-z]\s+', ' ', text)
    text = re.sub(r'^[a-z]\s+', ' ', text)
    text = re.sub(r'\s+', ' ', text)

    # Removendo números (opcional)
    text = re.sub(r'\d+', '', text)

    # Removendo quebra de linha manual
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\t', '', text)

    # remoção de stopwords, gera os radicais conforme a funçõa da palavra no contexto
    text = ' '.join([word for word in text.split() if word not in ENGLISH_STOP_WORDS ])


    return text


In [ ]:
#função para classificar a palavra conforme a função na frase
def get_wordnet_pos(word):
    """Mapeia a tag POS do NLTK para a primeira caracter do lematizador do WordNet."""
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


In [ ]:
df['txt_limpo'] = [preprocess(text) for text in df['text']]

In [ ]:
# Lematização, gera os radicias levando em consideração a função da palavra no texto
df['txt_lmtzd'] = df['txt_limpo'].apply(lambda x: ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(x)]))


In [ ]:
#Aplicação de limpeza de residuos pós lematização
df['lmtzd_limpo'] = [preprocess(text) for text in df['txt_lmtzd']]


In [ ]:
#remoção de espaços vazios
#substitui espaços vazios por nan
df['lmtzd_limpo'].replace('', np.nan, inplace=True)
# remoção de nans
df.dropna(subset=['lmtzd_limpo'],axis=0, inplace=True)

# remoção de ocorrências de frase com 3 palavras ou menos
df = df[df['lmtzd_limpo'].str.split().apply(len) >= 4]

# remoção de duplicatas
df.drop_duplicates(subset=['lmtzd_limpo'], inplace=True)

In [ ]:
alt = (df['lmtzd_limpo']==df['txt_lmtzd']).sum()
print(f"número de ocorrências que sofretam alteração {alt} - {df.shape[0]}") # dos 37431 do df_1 apenas 3001 e dos 3302 do df_2, apenas 994, não tiveram ajustes no novo processo de limpeza. 
# Isso demonstra que numa sequência de passos do pré-processamento, alguns passos devem ser repetidos
df.shape

In [ ]:
#excluindo colunas excedente usada no tratamento.
df['txt_lmtzd'] = df['lmtzd_limpo']
df.drop(['lmtzd_limpo'], axis=1, inplace = True)

In [ ]:
# obtendo rótulos individuais, após tratamento
y_df_all_names = df['subject']
y_df = df['target']

# Exibindo os rótulos convertidos
df_labels, df_sizes = np.unique(y_df, return_counts=True)
true_k_df = df_labels.shape[0]
print(f"{df.shape[0]} documentos - {true_k_df} categorias")

In [ ]:
df.head()

#### Fim da Limpeza

In [ ]:

df['c_count'] = df['txt_lmtzd'].str.len()
df['w_count'] = df['txt_lmtzd'].str.split().str.len()


In [ ]:
menor = len(df['txt_lmtzd'][df['w_count'].idxmin()].split())

In [ ]:
pd.set_option('display.max_colwidth', None)  # Evita truncamento de coluna
pd.set_option('display.max_rows', 15)  # Ajusta o número máximo de linhas a serem exibidas

# Exibindo as primeiras 15 ocorrências da coluna 'texto'
# print(df['txt_lmtzd'].head(15))

### Visualização dos dados após o tratamento

In [ ]:
# Agregando todos os textos para visualização
textos_df = " ".join(texto for texto in df['txt_lmtzd'])


In [ ]:
# Criando a nuvem de palavras
# wordcloud_df = WordCloud(background_color='white').generate(textos_df)
wordcloud_df = WordCloud(width=800, height=400, max_font_size=100, max_words=100, min_font_size=10,background_color='white').generate(textos_df)


In [ ]:
# Visualizando a nuvem de palavras
plt.imshow(wordcloud_df, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
wordcloud_df.to_file("df2_tratados.png")
print(f'arquivo df2_tratados.png criado')

#### Gerando os vetores - Vetorização

In [ ]:
# 1. Criando Representação Vetorial de Texto

In [ ]:
# 1.1 - TF-IDF
tfidf_vec = TfidfVectorizer(stop_words='english', max_df=0.2, min_df=0.1 , ngram_range=(1,3))
 
dfx_idf = tfidf_vec.fit_transform(df['txt_lmtzd'])



In [ ]:
dfx_idf.shape

In [ ]:
#  1.3 Word to Vector -  W2V
# Treinamento do modelo Word2Vec
model_df = Word2Vec([text.split() for text in df['txt_lmtzd']], vector_size=100, window=5, min_count=1, workers=4)


In [ ]:
# Função para converter texto em um vetor médio de word embeddings

def get_avg_word_vec(text, model):
    words = text.split()
    word_vecs = [model.wv[word] for word in words if word in model.wv]

    if len(word_vecs) == 0:
        return np.zeros(model.vector_size)
    elif len(word_vecs) == 1:
        return word_vecs[0]
    else:
        return np.mean(word_vecs, axis=0)

In [ ]:
# Aplicar a função em todos os textos

dfx_w2v = pd.DataFrame([get_avg_word_vec(text,model_df) for text in df['txt_lmtzd']])


In [ ]:
dfx_w2v.shape

#### Redução de dimensionalidade/ nornalização dos dados e Visualização

In [ ]:
#somente para Google Colab
#!pip install umap
# !pip install umap-learn

In [ ]:
import umap as umap
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Inicialize o objeto StandardScaler
# scaler = MinMaxScaler()
scaler = StandardScaler()



In [ ]:
#redução de dimensionaliade usando UMAP

In [ ]:
umap = umap.UMAP(random_state=42)

umap_tfidf = umap.fit_transform(dfx_idf)
umap_w2v = umap.fit_transform(dfx_w2v)

# Padronize os vetores
umap_tfidf_sc = scaler.fit_transform(umap_tfidf)
umap_w2v_sc = scaler.fit_transform(umap_w2v)

In [ ]:
# criar imagens dos gráficos dos dados reduzidos com os targets
# coloca na seção de resultado juntos com os dados após a aplicação do modelo.
# fazer isso com os escolhidos para a aplicação

In [ ]:
plt.figure(figsize=(15,10))

In [ ]:

plt.subplot(2, 2, 1)
plot_(2,umap_tfidf,'UMAP-TF-IDF',y_df)

plt.subplot(2, 2, 2)
plot_(2,umap_w2v,'UMAP-Word2Vec',y_df)

plt.subplot(2, 2, 3)
plot_(2,umap_tfidf_sc,'UMAP-TF-IDF_Norm',y_df)

plt.subplot(2, 2, 4)
plot_(2,umap_w2v_sc,'UMAP-Word2Vec_Norm',y_df)

plt.tight_layout()
plt.savefig('/images/df2_distrib_dados_UMAP.png')
plt.show()


In [ ]:
#Redução de dimensionalidade com SVD

In [ ]:

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=2)  # Escolha um número de componentes
svd_tfidf = svd.fit_transform(dfx_idf)
svd_w2v =   svd.fit_transform(dfx_w2v)

# Padronize os vetores
svd_tfidf_sc = scaler.fit_transform(svd_tfidf)
svd_w2v_sc = scaler.fit_transform(svd_w2v)


In [ ]:

plt.subplot(2,2,1)
plot_(2,svd_tfidf,'SVD-TF-IDF',y_df)

plt.subplot(2,2,2)
plot_(2,svd_w2v,'SVD-Word2Vec',y_df)

plt.subplot(2,2,3)
plot_(2,svd_tfidf_sc,'SVD-TF-IDF_Norm',y_df)

plt.subplot(2,2,4)
plot_(2,svd_w2v_sc,'SVD-Word2Vec_Norm',y_df)

plt.tight_layout()
plt.savefig('/images/df2_distrib_dados_SVD.png')
plt.show()



In [ ]:
# # Redução com NMF no texto ao TFIDF (não será raelizado ocom W2V)

In [ ]:

from sklearn.decomposition import NMF

nmf = NMF(n_components=2, random_state=0)  # n_components é o número de tópicos
nmf_tfidf = nmf.fit_transform(dfx_idf)

# Padronize os vetores
nmf_tfidf_sc = scaler.fit_transform(nmf_tfidf)


In [ ]:

plt.subplot(2,2,1)
plot_(2,nmf_tfidf,'NMF-TF-IDF',y_df)

plt.subplot(2,2,2)
plot_(2,nmf_tfidf_sc,'NMF-TF-IDF_norm',y_df)

plt.tight_layout()
plt.savefig('/images/df2_distrib_dados_NMF.png')
plt.show()

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=42)
tsne_tfidf = tsne.fit_transform(dfx_idf.toarray())
tsne_w2v   = tsne.fit_transform(dfx_w2v)

# Padronize os vetores
tsne_tfidf_sc = scaler.fit_transform(tsne_tfidf)
tsne_w2v_sc = scaler.fit_transform(tsne_w2v)

In [ ]:

# plt.subplot(2,2,1)
# plot_(2,tsne_tfidf,'tSEN-TF-IDF',y_df)

# plt.subplot(2,2,2)
# plot_(2,tsne_w2v,'tSEN-Word2Vec',y_df)


# plt.subplot(2,2,3)
# plot_(2,tsne_tfidf_sc,'tSEN-TF-IDF_Norm',y_df)

plt.subplot(2,2,4)
plot_(2,tsne_w2v_sc,'tSEN-Word2Vec_Norm',y_df)


plt.tight_layout()
plt.savefig('df2_distrib_dados_tsne.png')
plt.show()

## Modelo - Experimentação, treinamento, aplicação, validação e ajustes

### 3. Experimentação - melhor valor de K

###### 

======================= INICIO DO TESTE ==============================

In [ ]:
# import sklearn.cluster.KMeans

In [ ]:
# FAzer assim que terminar a geração do vetor
# Substituir pelas listas de vetores normalizados e fazer um for

Xs = ['nmf_tfidf_sc','svd_tfidf_sc','tsne_tfidf_sc','umap_tfidf_sc','svd_w2v_sc','tsne_w2v_sc','umap_w2v_sc']


# Variando os parâmetros do K-means
n_clusters_options = [2, 3, 4, 5, 6]
init_options = ['k-means++', 'random']
best_score = -1
best_params = {}

for n_clusters in n_clusters_options:
    print('==========')
    print(f'numero de clusters: {n_clusters}')
    for init in init_options:
        print(f'inicialização com: {init} \n-----')
        kmeans = KMeans(n_clusters=n_clusters, init=init, n_init=10, max_iter=300,  random_state=42)
        for x in Xs:
            # print(f'dataset: {x}')
            X = eval(x)
            kmeans.fit(X)
            labels = kmeans.labels_
            score = silhouette_score(X, labels)
            # print(f'Score: {score} \n')
            
            if score > best_score:
                best_score = score
                best_params = {'n_clusters': n_clusters, 'init': init, 'dataset':x}

print(f"Melhores parâmetros: {best_params}")
print(f"Melhor pontuação de silhueta: {best_score:.2f}")

###### ======================= FIM DO TESTE ==============================

In [ ]:
import pandas as pd

# Supondo que você tenha um DataFrame chamado df

# Ajustando opções de exibição do pandas para evitar truncamento
pd.set_option('display.max_colwidth', None)  # Evita truncamento de coluna
pd.set_option('display.max_rows', 15)  # Ajusta o número máximo de linhas a serem exibidas
print(df['text'].head(1))

In [ ]:
# Criando dataset para facilitar a geração de estatísticas dos resultados

# sil_avg_df = pd.DataFrame(columns=['dataset', 'tokenizador', 'n_clusters', 'score'])
sil_avg_df = pd.DataFrame(columns=['dataset', 'tokenizador','redutor', 'n_clusters', 'sil_score','inertia'])



In [ ]:
from datetime import datetime

# Imprimir a hora atual
print('Inicio:', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
k = 3
n_init = 10
max_iter = 300
tol = 1e-4
# metric = "cosine"  @# não é possível configura a métrica na biblioteca do sklearn, há uma otimiação interna 
init = "k-means++"
weight  = None
random_state = 42


In [ ]:
# início
print('Inicio:', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
# Identificando o melhor valor de k
# ajustar para 2 até 7 ou 9, reduzir o númeor de experimentos.
# Adiciona os valores num dataframe para medição posterior
range_n_clusters = range(2,10) #(2,20)
# Definindo os parâmetros

clbs_it = pd.DataFrame()
# sil_avg_df = pd.DataFrame(columns=['dataset', 'tokenizador','redutor', 'n_clusters', 'score'])


it = 0
for num_clusters in range_n_clusters:
    print('iteração:', it, ' : ',datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    it = it+1
    km = KMeans(n_clusters=num_clusters, 
                init=init, 
                n_init=n_init, 
                max_iter=max_iter, 
                tol=tol, 
                random_state=random_state)
    # km = KMeans(n_clusters=num_clusters, init='k-means++', n_init=10, max_iter=300, random_state=42)
  

    
    # Com vetor TF-IDF:
    # print('TF-IDF')
    # SVD:
    
    df_xcluster_labels = km.fit_predict(svd_tfidf_sc)
    lst2 = ['svd_tfidf_sc','tfidf','SVD',num_clusters,silhouette_score(svd_tfidf_sc, df_xcluster_labels),km.inertia_]
    sil_avg_df.loc[len(sil_avg_df)+1] = lst2
    clbs_it['svd_tfidf_sc-' +f'{num_clusters}'] = df_xcluster_labels 
    
    # UMAP:
    df_xcluster_labels = km.fit_predict(umap_tfidf_sc)
    lst2 = ['umap_tfidf_sc','tfidf','UMAP',num_clusters,silhouette_score(umap_tfidf_sc, df_xcluster_labels),km.inertia_]
    sil_avg_df.loc[len(sil_avg_df)+1] = lst2
    clbs_it['umap_tfidf_sc-' +f'{num_clusters}'] = df_xcluster_labels 

    # t-SNE
    df_xcluster_labels = km.fit_predict(tsne_tfidf_sc)
    lst2 = ['tsne_tfidf_sc','tfidf','t-SNE',num_clusters,silhouette_score(tsne_tfidf_sc, df_xcluster_labels),km.inertia_]
    sil_avg_df.loc[len(sil_avg_df)+1] = lst2    
    clbs_it['tsne_tfidf_sc-' +f'{num_clusters}'] = df_xcluster_labels 

    # NMF    :
    df_xcluster_labels = km.fit_predict(nmf_tfidf_sc)
    lst2 = ['nmf_tfidf_sc','tfidf','NMF',num_clusters,silhouette_score(nmf_tfidf_sc, df_xcluster_labels),km.inertia_]
    sil_avg_df.loc[len(sil_avg_df)+1] = lst2   
    clbs_it['nmf_tfidf_sc-' +f'{num_clusters}'] = df_xcluster_labels 



    # Com vetor Word to Vector W2v:
    # print('W2v')
    # SVD:
    df_xcluster_labels = km.fit_predict(svd_w2v_sc)
    lst2 = ['svd_w2v_sc','W2v','SVD',num_clusters,silhouette_score(svd_w2v_sc, df_xcluster_labels),km.inertia_]
    sil_avg_df.loc[len(sil_avg_df)+1] = lst2
    clbs_it['svd_w2v_sc-' +f'{num_clusters}'] = df_xcluster_labels 
    
    # UMAP:
    df_xcluster_labels = km.fit_predict(umap_w2v_sc)
    lst2 = ['umap_w2v_sc','W2v','UMAP',num_clusters,silhouette_score(umap_w2v_sc, df_xcluster_labels),km.inertia_]
    sil_avg_df.loc[len(sil_avg_df)+1] = lst2
    clbs_it['umap_w2v_sc-' +f'{num_clusters}'] = df_xcluster_labels 

    # t-SNE
    df_xcluster_labels = km.fit_predict(tsne_w2v_sc)
    lst2 = ['tsne_w2v_sc','W2v','t-SNE',num_clusters,silhouette_score(tsne_w2v_sc, df_xcluster_labels),km.inertia_]
    sil_avg_df.loc[len(sil_avg_df)+1] = lst2   
    clbs_it['tsne_w2v_sc-' +f'{num_clusters}'] = df_xcluster_labels 

    # NMF    : Não aplicável à W2v
     
    

In [ ]:
sil_avg_df.head()

In [ ]:
# fim
print('termino: ', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
#Localizando o melhor valor para k
n = np.argmax(sil_avg_df[['sil_score']])
best_k = sil_avg_df.iloc[n]['n_clusters']
sil_avg_df.iloc[n]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

def kmeans_elbow_curve(data, k_range, n_init, title):
  """
  Gera a curva de cotovelo para o algoritmo k-means.

  Argumentos:
    data: Matriz numpy com os dados.
    k_range: Lista com os valores de k a serem testados.
    n_init: Número de inicializações aleatórias dos centroides para cada valor de k.
    title: Título do gráfico.

  Retorna:
    Tuple com:
      - k_values: Lista com os valores de k.
      - distortions: Lista com as distorções médias para cada valor de k.
  """

  k_values = []
  distortions = []
  for k in k_range:
    k_values.append(k)
    
    distortion_list = []
    for _ in range(n_init):
      kmeans = KMeans(n_clusters=k,n_init=10, random_state=np.random.randint(0, 100000))
      kmeans.fit(data)
      distortion_list.append(kmeans.inertia_)
    
    distortions.append(np.mean(distortion_list))

  # Plotando a curva de cotovelo
  plt.plot(k_values, distortions)
  plt.xlabel("Número de clusters (k)")
  plt.ylabel("Distorção média")
  plt.title(title)
  plt.show()

# # Definindo os datasets
# svd_tfidf = ... # Insira o dataset svd_tfidf aqui
# svd_W2v = ... # Insira o dataset svd_W2v aqui

# Definindo os parâmetros
k_range = list(range(2, 11))
n_init = 10

# # Gerando as curvas de cotovelo
# kmeans_elbow_curve(svd_tfidf, k_range, n_init, "Curva de cotovelo - SVD TF-IDF")
# kmeans_elbow_curve(svd_W2v, k_range, n_init, "Curva de cotovelo - SVD Word2Vec")

# Adicionando mais datasets
# ... Insira o código para adicionar mais datasets aqui ...

# Função para plotar as curvas de cotovelo de vários datasets
def plot_multiple_elbow_curves(data_dict, title):
  """
  Plota as curvas de cotovelo para vários datasets.

  Argumentos:
    data_dict: Dicionário com os datasets. As chaves do dicionário devem ser strings com os nomes dos datasets e os valores devem ser matrizes numpy com os dados.
    title: Título do gráfico.
  """

  for name, data in data_dict.items():
    kmeans_elbow_curve(data, k_range, n_init, name)
  
  plt.xlabel("Número de clusters (k)")
  plt.ylabel("Distorção média")
  plt.title(title)
  name = f'{dataset}-{tokenizador}-{redutor}-elbow_sil_df2.png'
  plt.savefig(name)  
  plt.show()

In [ ]:
# Plotando as curvas de cotovelo dos datasets
data_dict = {
    "SVD TF-IDF": np.array(svd_tfidf),
    "UMAP TF-IDF": umap_tfidf,
    "tSEN TF-IDF": tsne_tfidf,
    "NMF TF-IDF": nmf_tfidf,
    "SVD Word2Vec": np.array(svd_w2v),
    "UMAP Word2Vec": umap_w2v,
    "tSEN Word2Vec": tsne_w2v
    
  # ... Insira outros datasets aqui ...
}

plot_multiple_elbow_curves(data_dict, "Curvas de cotovelo - Múltiplos datasets")

In [ ]:

# Plotando a análise de silhueta dos datasets
data_dict = {
    "UMAP TF-IDF": umap_tfidf,
    "tSEN TF-IDF": tsne_tfidf,
    "NMF TF-IDF": nmf_tfidf,
    "SVD TF-IDF": svd_tfidf,    
    "SVD Word2Vec": svd_w2v,
    "UMAP Word2Vec": umap_w2v,
    "tSEN Word2Vec": tsne_w2v
    
  # ... Insira outros datasets aqui ...
}

# plot_multiple_silhouette_analysis(data_dict, "Análise de Silhueta - Múltiplos datasets")

In [ ]:
# fim
print('termino: ', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:

# Excluir possíveis duplicatas das combinações 
print(sil_avg_df.shape)
combinacoes_unicas = sil_avg_df[['dataset', 'tokenizador', 'redutor']].drop_duplicates()
print(combinacoes_unicas.shape)


# Plotar gráficos para cada combinação
for _, row in combinacoes_unicas.iterrows():
    dataset = row['dataset']
    tokenizador = row['tokenizador']
    redutor = row['redutor']
    
    # Filtrar o DataFrame para a combinação atual
    df_filtrado = sil_avg_df[(sil_avg_df['dataset'] == dataset) & (sil_avg_df['tokenizador'] == tokenizador) & (sil_avg_df['redutor'] == redutor)]
    plt.figure(figsize=(12, 5))
    plt.suptitle(f'df2: {tokenizador} e {redutor}', fontsize=16, y=1.05)  # Ajuste o tamanho da fonte e a posição do título conforme necessário
    

    
    # Plotar Silhouette Score

    plt.subplot(1, 2, 1) # Subplot 1
    plt.plot(df_filtrado['n_clusters'], df_filtrado['sil_score'], marker='o')
    plt.title(f'Silhouette Score')#\n{tokenizador}, {redutor}')
    plt.xlabel('Número de Clusters')
    plt.ylabel('Silhouette Score')
    plt.grid(True)
    
    # Plotar Inertia
    plt.subplot(1, 2, 2) # Subplot 2
    plt.plot(df_filtrado['n_clusters'], df_filtrado['inertia'], marker='o')
    plt.title(f'Inertia')#\n{tokenizador}, {redutor}')
    plt.xlabel('Número de Clusters')
    plt.ylabel('Inertia')
    plt.grid(True)
    
    plt.tight_layout()
    name = f'df2_{tokenizador}-{redutor}-elbow_sil.png'
    plt.savefig(name)
    plt.show();

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Suponha que sil_avg_df seja o DataFrame contendo os dados

# Encontrar combinações únicas de 'dataset', 'tokenizador', 'redutor' para usar como categorias
combinacoes = sil_avg_df[['dataset', 'tokenizador', 'redutor']].drop_duplicates()

# Criar uma figura e um eixo para o plot
fig, ax = plt.subplots(figsize=(14, 8))

# Cores diferentes para cada combinação
cores = plt.cm.tab20(np.linspace(0, 1, len(combinacoes)))

for i, (index, row) in enumerate(combinacoes.iterrows()):
    # Filtrar o DataFrame para a combinação atual
    df_filtrado = sil_avg_df[(sil_avg_df['dataset'] == row['dataset']) & 
                             (sil_avg_df['tokenizador'] == row['tokenizador']) & 
                             (sil_avg_df['redutor'] == row['redutor'])]
    
    # Ordenar o DataFrame filtrado por 'n_clusters' para garantir a linha contínua
    df_filtrado = df_filtrado.sort_values(by='n_clusters')
    
    # Plotar linha e pontos
    ax.plot(df_filtrado['n_clusters'], df_filtrado['sil_score'], marker='o', linestyle='-', color=cores[i], label=f"{row['dataset']}, {row['tokenizador']}, {row['redutor']}")
    
# Ajustar detalhes do gráfico
ax.set_xlabel('Número de Clusters')
ax.set_ylabel('Silhouette Score')
ax.set_title('Silhouette Scores por Número de Clusters para Diferentes Configurações')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.xticks(np.unique(sil_avg_df['n_clusters'])) # Garante que todos os números de clusters estejam nos ticks
plt.grid(True)
plt.tight_layout()
plt.savefig('/images/df2_silhueta_numClusters')
plt.show()



In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Suponha que sil_avg_df seja o DataFrame contendo os dados

# Encontrar combinações únicas de 'dataset', 'tokenizador', 'redutor' para usar como categorias
combinacoes = sil_avg_df[['dataset', 'tokenizador', 'redutor']].drop_duplicates()

# Criar uma figura e um eixo para o plot
fig, ax = plt.subplots(figsize=(14, 8))

# Cores diferentes para cada combinação
cores = plt.cm.tab20(np.linspace(0, 1, len(combinacoes)))

for i, (index, row) in enumerate(combinacoes.iterrows()):
    # Filtrar o DataFrame para a combinação atual
    df_filtrado = sil_avg_df[(sil_avg_df['dataset'] == row['dataset']) & 
                             (sil_avg_df['tokenizador'] == row['tokenizador']) & 
                             (sil_avg_df['redutor'] == row['redutor'])]
    
    # Ordenar o DataFrame filtrado por 'n_clusters' para garantir a linha contínua
    df_filtrado = df_filtrado.sort_values(by='n_clusters')
    
    # Plotar linha e pontos para a inércia
    ax.plot(df_filtrado['n_clusters'], df_filtrado['inertia'], marker='o', linestyle='-', color=cores[i], label=f"{row['dataset']}, {row['tokenizador']}, {row['redutor']}")

# Ajustar detalhes do gráfico
ax.set_xlabel('Número de Clusters')
ax.set_ylabel('Inércia')  # Alterar para Inércia
ax.set_title('Método de Cotovelo: Inércia por Número de Clusters para Diferentes Configurações')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.xticks(np.unique(sil_avg_df['n_clusters'])) # Garante que todos os números de clusters estejam nos ticks
plt.grid(True)
plt.tight_layout()
plt.savefig('/images/df2_cotovelo_numClusters')
plt.show()




In [ ]:
# eval(sil_avg_df['dataset'])
clusters_unique = sil_avg_df['n_clusters'].unique()
datasets_unique = sil_avg_df['dataset'].unique()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.metrics import silhouette_samples, silhouette_score
import pandas as pd

# Supondo que sil_avg_df seja o seu DataFrame com os dados de clusters e os rótulos de cluster correspondentes
# E que você tenha seu conjunto de dados original X para calcular as amostras de silhueta

# Escolher um esquema de cores para os gráficos
cmap = cm.get_cmap("Spectral")

# Definindo o número de colunas por linha
cols_per_row = 4

# Estrutura para armazenar informações dos gráficos (incluindo a presença de valores negativos)
plot_info = []

for dataset in datasets_unique:
    for n_clusters in clusters_unique:
        # Assumindo que 'clbs_it' e 'dataset' sejam definidos anteriormente (ajuste conforme necessário)
        cluster_labels = clbs_it[f'{dataset}-{n_clusters}']
        X = eval(dataset)  # Use com cautela, certifique-se de que 'dataset' vem de uma fonte confiável
        silhouette_vals = silhouette_samples(X, cluster_labels)
        
        # Verifica se existem valores negativos
        has_negative_values = np.any(silhouette_vals < 0)
        
        # Armazenar informações para ordenação posterior
        plot_info.append({
            "dataset": dataset,
            "n_clusters": n_clusters,
            "silhouette_vals": silhouette_vals,
            "cluster_labels": cluster_labels,
            "has_negative_values": has_negative_values
        })

# Ordenar os gráficos colocando aqueles sem valores negativos primeiro
plot_info_sorted = sorted(plot_info, key=lambda x: x['has_negative_values'])

# Calcular o número total de subplots
total_plots = len(plot_info_sorted)

# Calcular o número de linhas necessário, garantindo pelo menos uma linha
num_rows = ((total_plots - 1) // cols_per_row) + 1

# Ajustar o tamanho da figura para acomodar os subplots adequadamente
fig_width = 14
fig_height = num_rows * 4  # Ajuste a altura conforme necessário para evitar sobreposição
plt.figure(figsize=(fig_width, fig_height))

for i, info in enumerate(plot_info_sorted):
    ax = plt.subplot(num_rows, cols_per_row, i + 1)
    plt.subplots_adjust(bottom=0.1, top=0.9, hspace=0.4, wspace=0.4)

    n_clusters = info["n_clusters"]
    silhouette_vals = info["silhouette_vals"]
    cluster_labels = info["cluster_labels"]
    sil_avg = silhouette_score(X, cluster_labels)
    
    y_lower = 10
    for j in range(n_clusters):
        jth_cluster_silhouette_vals = silhouette_vals[cluster_labels == j]
        jth_cluster_silhouette_vals.sort()
        
        size_cluster_j = jth_cluster_silhouette_vals.shape[0]
        y_upper = y_lower + size_cluster_j

        color = cmap(float(j) / n_clusters)
        ax.fill_betweenx(np.arange(y_lower, y_upper),
                         0, jth_cluster_silhouette_vals,
                         facecolor=color, edgecolor=color, alpha=0.7)
        
        y_lower = y_upper + 10  # 10 para o espaço entre silhuetas

    ax.axvline(x=sil_avg, color="red", linestyle="--")
    ax.set_yticks([])  # Limpar os yaxis labels / ticks
    ax.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    ax.set_title(f"{info['dataset']} | {n_clusters} clusters", fontsize=10)
    ax.set_xlabel('Silhouette coefficient values', fontsize=9)
    ax.set_ylabel('Cluster', fontsize=9)

plt.tight_layout()
plt.show()


In [ ]:
# Encontrar a entrada com o maior Silhouette Score
melhor_silhouette = sil_avg_df.loc[sil_avg_df['sil_score'].idxmax()]
print(f"Melhor Silhouette Score: {melhor_silhouette['sil_score']} para {melhor_silhouette['n_clusters']} clusters, com tokenizador {melhor_silhouette['tokenizador']} e redutor {melhor_silhouette['redutor']}")



##### 4. Treinamento do KMeans com o Texto Tratado

In [ ]:
df.shape

In [ ]:
#treinando com os melhores parâmetros
# best_xkdf

km_2kp = KMeans(n_clusters=2,init='k-means++', n_init=10, max_iter=300, random_state=42)
km_4kp = KMeans(n_clusters=4,init='k-means++', n_init=10, max_iter=300, random_state=42)
# kmeans = KMeans(n_clusters=k, init='k-means++', n_init=10, max_iter=300, random_state=42)



In [ ]:
#Submetendo o Kmean aos parâmetros]
#tfidf
xcl_tf_svd = km_2kp.fit_predict( svd_tfidf_sc)#dfx_idf)
xcl_tf_nmf = km_2kp.fit_predict( nmf_tfidf_sc)#dfx_idf)
xcl_tf_umap = km_2kp.fit_predict(umap_tfidf_sc)#dfx_idf)
xcl_tf_tsne = km_2kp.fit_predict(tsne_tfidf_sc)#dfx_idf)

#W2v
xcl_w2v_svd = km_2kp.fit_predict( svd_w2v_sc)#dfx_idf)
xcl_w2v_umap = km_2kp.fit_predict(umap_w2v_sc)#dfx_idf)
xcl_w2v_tsne = km_2kp.fit_predict(tsne_w2v_sc)#dfx_idf)
# umap_w2v_sc,tsne_w2v_sc,svd_w2v_sc

In [ ]:
#Submetendo o Kmean aos parâmetros]
#tfidf
xcl_tf_svd = km_4kp.fit_predict( svd_tfidf_sc)#dfx_idf)
xcl_tf_nmf = km_4kp.fit_predict( nmf_tfidf_sc)#dfx_idf)
xcl_tf_umap = km_4kp.fit_predict(umap_tfidf_sc)#dfx_idf)
xcl_tf_tsne = km_4kp.fit_predict(tsne_tfidf_sc)#dfx_idf)

#W2v
xcl_w2v_svd = km_4kp.fit_predict( svd_w2v_sc)#dfx_idf)
xcl_w2v_umap = km_4kp.fit_predict(umap_w2v_sc)#dfx_idf)
xcl_w2v_tsne = km_4kp.fit_predict(tsne_w2v_sc)#dfx_idf)
# umap_w2v_sc,tsne_w2v_sc,svd_w2v_sc

In [ ]:
# print(xcl_tf_svd,  xcl_tf_nmf,xcl_tf_umap,xcl_tf_tsne)
df['xcl_tf_svd']=xcl_tf_svd
df['xcl_tf_nmf']=xcl_tf_nmf
df['xcl_tf_umap']=xcl_tf_umap
df['xcl_tf_tsne']=xcl_tf_tsne

df['xcl_w2v_svd']=xcl_w2v_svd
df['xcl_w2v_umap']=xcl_w2v_umap
df['xcl_w2v_tsne']=xcl_w2v_tsne

# df.head()

In [ ]:
df_cl = df[['xcl_tf_svd','xcl_tf_nmf','xcl_tf_umap','xcl_tf_tsne','xcl_w2v_svd','xcl_w2v_umap','xcl_w2v_tsne']]
df_cl.head()

In [ ]:
Xs = ['tsne_tfidf_sc', 'svd_tfidf_sc','umap_tfidf_sc','nmf_tfidf_sc','tsne_w2v_sc','svd_w2v_sc','umap_w2v_sc']
cl=  ['xcl_tf_tsne'  , 'xcl_tf_svd'  ,'xcl_tf_umap'  ,'xcl_tf_nmf'  ,'xcl_w2v_tsne','xcl_w2v_svd','xcl_w2v_umap']

In [ ]:
for x,y in zip(Xs,cl):
    data = eval(x)
    labels = eval(y)
    plot_(2,data,x,labels)
    name = 'df2_' + x + '.png'
    plt.savefig(name)
    plt.show()
# plt.subplot(2,2,1)
# plot_(2,tsne_tfidf,'tSEN-TF-IDF',y_df)

# plt.subplot(2,2,2)
# plot_(2,tsne_w2v,'tSEN-Word2Vec',y_df)


# plt.subplot(2,2,3)
# plot_(2,tsne_tfidf_sc,'tSEN-TF-IDF_Norm',y_df)

# plt.subplot(2,2,4)
# plot_(2,tsne_w2v_sc,'tSEN-Word2Vec_Norm',y_df)

In [ ]:
# # Crie um gráfico de dispersão dos dados orinagis
# import plotly.express as px

# fig = px.scatter(dfx_idf, x=dfx_idf[:, 0], y=dfx_idf[:, 1], color=df['target'])
# fig.update_traces(marker=dict(size=3)) 

# # Adicione um título ao gráfico
# fig.update_layout(title='K-means clustering' + " - " + x)

# # Adicione rótulos aos eixos x e y
# fig.update_xaxes(title='Feature 1')
# fig.update_yaxes(title='Feature 2')

# # Mostre o gráfico
# fig.show()

In [ ]:
import pandas as pd

# Supondo que df seja o seu DataFrame


for x,y in zip(Xs,cl):
    # Criando o dicionário para armazenar as contagens
    distribution = {}
    
    # Agrupando por 'target' e contando as ocorrências de 'y_pred'
    for target, group in df.groupby('target'):
        distribution[target] = group[y].value_counts().to_dict()
    
    # Exibindo a distribuição
    print(distribution)

# Xs = ['tsne_tfidf_sc', 'svd_tfidf_sc','umap_tfidf_sc','nmf_tfidf_sc','tsne_w2v_sc','svd_w2v_sc','umap_w2v_sc']
# cl=  ['xcl_tf_tsne'  , 'xcl_tf_svd'  ,'xcl_tf_umap'  ,'xcl_tf_nmf'  ,'xcl_w2v_tsne','xcl_w2v_svd','xcl_w2v_umap']

In [ ]:
# Gráfico de dispersão dos resultados

import plotly.express as px


for x,y in zip(Xs,cl):
    data = eval(x)
    labels = eval(y)
# Crie um gráfico de dispersão
    fig = px.scatter(data, x=data[:, 0], y=data[:, 1], color=labels)
    fig.update_traces(marker=dict(size=3)) 
    
    # Adicione um título ao gráfico
    fig.update_layout(title='K-means clustering' + " - " + x)
    
    # Adicione rótulos aos eixos x e y
    fig.update_xaxes(title='Feature 1')
    fig.update_yaxes(title='Feature 2')
    
    # Mostre o gráfico
    fig.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Supondo que 'Xs' e 'cl' sejam listas de strings com os nomes das variáveis
# Exemplo:
# Xs = ['svd_w2v_sc_1', 'svd_w2v_sc_2']
# cl = ['labels_1', 'labels_2']

for x, y in zip(Xs, cl):
    data = eval(x)
    labels = eval(y)
    
    # Crie um gráfico de dispersão
    plt.figure(figsize=(10, 8))
    
    # Obter os rótulos únicos para colorir os pontos
    unique_labels = np.unique(labels)
    for label in unique_labels:
        # Filtrar pontos por rótulo
        ix = np.where(labels == label)
        plt.scatter(data[ix, 0], data[ix, 1], label=f'Cluster {label}', s=2)  # s controla o tamanho do ponto
    
    # Adicione um título ao gráfico
    plt.title(f'K-means clustering - {x}')
    
    # Adicione rótulos aos eixos x e y
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    
    # Adicionar legenda
    plt.legend()
    
    # Mostrar o gráfico
    plt.show()


In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, silhouette_score
from sklearn.preprocessing import LabelEncoder


metrics = {}
# cluster_ = ['xcl_tf_tsne', 'xcl_tf_svd', 'xcl_tf_nmf', 'xcl_tf_umap']
cluster_ = ['xcl_tf_tsne'  
            ,'xcl_tf_svd'  
            ,'xcl_tf_nmf'  
            ,'xcl_tf_umap' 
            ,'xcl_w2v_svd' 
            ,'xcl_w2v_umap'
            ,'xcl_w2v_tsne']
# Calcula métricas para cada conjunto de clusters
for c in cluster_:
    print('*==================================*')
    cluster_col = f'{c}'
    print(f'cluster_col: {cluster_col}')
    
    # Matriz de confusão
    cm = confusion_matrix(df['target'], df[cluster_col])
    print(f"Matriz de confusão para {cluster_col}:\n", cm)
    
    # Relatório de classificação
    report = classification_report(df['target'], df[cluster_col], output_dict=True)
    print(f"Relatório de classificação para {cluster_col}:\n", report)
    
    # Calcula a pontuação de silhueta
    silhouette_avg = silhouette_score(df.drop(['text','subject','txt_limpo','txt_lmtzd','target', 'xcl_tf_tsne', 'xcl_tf_svd', 'xcl_tf_nmf', 'xcl_tf_umap'], axis=1), df[cluster_col])
     # silhouette_avg = silhouette_score(df.drop(['target', 'subject', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4'], axis=1), df[cluster_col])
    print(f"Pontuação de silhueta para {cluster_col}: {silhouette_avg}\n")
    
    metrics[cluster_col] = {
        'silhouette_score': silhouette_avg,
        **report['weighted avg']
    }

# Comparando os resultados
for cluster_col, metric in metrics.items():
    print(f"\n{cluster_col}: Silhouette Score = {metric['silhouette_score']}, \nPrecision = {metric['precision']}, \nRecall = {metric['recall']}, \nF1-Score = {metric['f1-score']}")


In [ ]:

#com Word2Vec - Lista
df_xcl_W2V = km_4kp.fit_predict(dfx_w2v)#xdf_W2V)
# df_lcluster_labels_W2V = km_4kp.fit_predict(kmW2V_ldf)

# antes de executar novamente, transformar a lista de 44898 elementos, cada um é uma np.array de 200 itens.
# preciso transformar ou num arrau de 44898 x 200 ou num dataframe

##### 5. Geração de Estatísticas e Mensuração dos Resultados

In [ ]:
# # len(df_xcl_itf)
# 'xcl_tf_tsne'  
#             ,'xcl_tf_svd'  
#             ,'xcl_tf_nmf'  
#             ,'xcl_tf_umap' 
#             ,'xcl_w2v_svd' 
#             ,'xcl_w2v_umap'
#             ,'xcl_w2v_tsne

In [ ]:
# cluster_labels = km_df.labels_
# # Aqui você pode calcular ARI, Silhouette Score etc., como mencionado anteriormente
# print(len(sdf_tfidf), '\n', np.unique(cluster_labels) ,'\n' , df_xcluster_labels ) #df1_xcluster_labels não pode se usado, foi iterado com 20 ciclos
# print(len(sdf_tfidf), '\n', np.unique(cluster_labels) ,'\n' , df_xcluster_labels )

# #kmW2V_xdf1 valor inicialmente usado,

In [ ]:
clbs_it[:3]

In [ ]:
matrizes = {'xcl_itf_tsne'  :xcl_tf_tsne
            ,'xcl_itf_svd'  :xcl_tf_svd 
            ,'xcl_itf_nmf'  :xcl_tf_nmf 
            ,'xcl_itf_umap' :xcl_tf_umap
            ,'xcl_w2v_svd' :xcl_w2v_svd
            ,'xcl_w2v_umap':xcl_w2v_umap
            ,'xcl_w2v_tsne':xcl_w2v_tsne
           }
# matrizes = {
#     "df_xcl_itf": df_xcl_tf,
#     "df_xcl_W2V": df_xcl_W2V
# }



In [ ]:
# Obtendo os três primeiros caracteres de cada chave (nome da variável)
primeiros_tres_caracteres = [nome[4:7] for nome in matrizes]

In [ ]:

set(y_df)
primeiros_tres_caracteres

In [ ]:
#filtrar as labels do kmeans para fazer a medição
metricas=[]


for nome,labels in matrizes.items():
    name=(nome[4:])
    labels = labels
    y = y_df
  # y = globals()[ 'y_' + nome[:2]]
    metricas.append(coleta_metricas(y,labels,name))




In [ ]:
df_metricas =  pd.DataFrame(metricas)
df_metricas.rename(columns={0:'dataset',1:'metrica',2:'score'}, inplace=True)

In [ ]:
df_metricas

In [ ]:
df_metricas.dtypes

In [ ]:
df_metricas.groupby(['dataset','metrica']).max(['score'])


In [ ]:
# Encontrar o array com o maior score
metricas = [array for array in metricas if array and array[2] is not None]
max_array = max(metricas, key=lambda x: x[2])

In [ ]:
#Exibir os resultados:
# meu dicionário tem dataset, metrica, score da métrica
# minhas métricas estão numa lista com todos os dataset trabalhados com td-idf e w2v.



In [ ]:
# from sklearn.metrics import adjusted_rand_score, f1_score, precision_score, recall_score, homogeneity_completeness_v_measure
# import numpy as np

# # Supondo que df_cluster_labels e y_df são seus clusters e rótulos para o primeiro conjunto de dados
# # Para o Índice Rand Ajustado
# ari_score_xdf = adjusted_rand_score(y_df, df_xcluster_labels)

# # Para V-measure
# homogeneity, completeness, v_measure = homogeneity_completeness_v_measure(y_df, df_xcluster_labels)

# # Para F1-Score, Precisão e Revocação (necessário adaptar para multi-classe se for o caso)
# # Supondo que os rótulos e clusters estão alinhados (caso contrário, será necessário alinhá-los)
# precision_df = precision_score(y_df, df_xcluster_labels, average='macro')
# recall_df = recall_score(y_df, df_xcluster_labels, average='macro')
# f1_score_df = f1_score(y_df, df_xcluster_labels, average='macro')

# # Repita o processo para df_cluster_labels e y_df

# # Exibindo os resultados
# print("ARI Score df:", ari_score_xdf)
# print("Homogeneity df:", homogeneity)
# print("Completeness df:", completeness)
# print("V-measure df:", v_measure)
# print("Precision df:", precision_df)
# print("Recall df:", recall_df)
# print("F1 Score df:", f1_score_df)

# # Repita a impressão para o segundo conjunto de dados


6. Visualizações, Incluindo Visualizações Gráficas dos Resultados

In [ ]:
#Aplicação do tSNE -
# ainda não testado
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Suponha que df['text_alg1'] e df['text_alg2'] sejam suas colunas de texto tratadas
# tfidf_mtx_ldf

# Aplicando t-SNE
# tsne = TSNE(n_components=2, random_state=42)
# tsne = TSNE(n_components=3, random_state=42)
# tsne_res_alg2 = tsne.fit_transform(dfx_idf.toarray())




In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=3, random_state=42)
tsne_res_alg2 = tsne.fit_transform(dfx_idf.toarray())

In [ ]:
# Visualização
plt.figure(figsize=(12, 6))


# plt.subplot(1, 2, 2)
# plt.scatter(tsne_res_alg2[:, 0], tsne_res_alg2[:, 1], alpha=0.5)
# plt.title('Tratamento - df')

# plt.show()

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(tsne_res_alg2[:, 0], tsne_res_alg2[:, 1], tsne_res_alg2[:, 2], c=y)

ax.set_title('t-SNE 3D')
plt.show()

#### Aplicando Kmeans

In [ ]:
# configuração dos Parâmetros
n_clusters = 4
distance_metric = "cosine" #  euclidean
n_init = 0
max_iter = 300
# df1
# gerador de vetores: Word to Vector w2v
# seleção de atributos: t-SNE
# df: tsne_w2v_sc
# método de escolha: cotovelo


# df2
# gerador de vetores: TF-IDF
# seleção de atributos: t-SNE
# df: tsne_tfidf_sc
# método de escolha: cotovelo

In [ ]:
# Aplicação do k-means
# Configurando o modelo
km_xdf = KMeans(n_clusters=n_clusters 
                ,n_init='auto'
                ,max_iter=max_iter
                ,random_state=42).fit(tsne_tfidf_sc) 
                # ,Preprocessing=True


In [ ]:
type(km_xdf.labels_)

In [ ]:
# Adicionar os rótulos dos clusters ao DataFrame
df['cluster_x'] = km_xdf.labels_
# df[['subject','target','cluster_x']]


In [ ]:
# df.head()

In [ ]:
df.groupby(['cluster_x'])['target'].count()

In [ ]:
df.groupby(['target'])['cluster_x'].count()

In [ ]:
# df[df['target']==1]['text']

In [ ]:
df_W2V_lbs = km_xdf.labels_


In [ ]:
km_xdf = KMeans(n_clusters=n_clusters,n_init=10, random_state=42).fit(dfx_idf)
df_idf_lbs_scl = km_xdf.labels_

In [ ]:
df_idf_lbs_scl

In [ ]:
type(km_xdf.labels_)
km_xdf.labels_.shape
dfx_w2v.shape

In [ ]:
# Adicionar os rótulos dos clusters ao DataFrame
df['cluster_x'] = km_xdf.labels_




In [ ]:
df[['subject','target','cluster_x']]
df.groupby(['subject','cluster_x'])['target'].count()


In [ ]:
# Cálculo das métricas e visualização dos resultados

In [ ]:
df.head(1)

In [ ]:
import numpy as np
from sklearn.metrics import f1_score, v_measure_score, adjusted_rand_score

# Definindo os rótulos reais e os grupos do KMeans
y_true = df['target']
y_pred = df['cluster_x']

# Calculando as métricas
f1 = f1_score(y_true, y_pred, average='weighted')
v_measure = v_measure_score(y_true, y_pred)
ari = adjusted_rand_score(y_true, y_pred)

# Imprimindo os resultados
print(f"F1-score: {f1:.3f}")
print(f"V-measure: {v_measure:.3f}")
print(f"ARI: {ari:.3f}")



In [ ]:
# Plotando as métricas
import matplotlib.pyplot as plt

plt.bar(['F1-score', 'V-measure', 'ARI'], [f1, v_measure, ari])
plt.xlabel('Métrica')
plt.ylabel('Valor')
plt.title('Métricas de Agrupamento')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Definindo as cores dos grupos
# colors = ['r', 'g', 'b', 'c', 'm']
colors = ['r', 'g', 'b', 'c', 'm']

# Plotando os pontos
plt.scatter(tsne_tfidf_sc[:, 0], tsne_tfidf_sc[:, 1],  c=df['cluster_x'],  label=f'Cluster {colors}', s=3)
# color=colors[df['cluster_x']])
 # color=colors[df['kmeans_group'][i]]

# Adicionando título e rótulos aos eixos
plt.title('Gráfico de Dispersão - Agrupamentos KMeans')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')

# Mostrando o gráfico
plt.savefig('df2_x_cluster.png')
plt.show()